### 分析2月份美团的数据，需要对大类名称进行整理规整，然后列出其单个成本、菜品收入、收入占比、销售占比 ###


In [1]:
 import pandas as pd
import numpy as np
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [2]:
original_data = pd.read_csv("C:\\Users\\86130\\Desktop\\meituan_data\\美团_3月.csv",encoding='ANSI')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte

In [ ]:
original_data.head()

## 评估数据 ##

In [ ]:
original_data.sample(10)

### 分析表格存在的问题： ###
### 1.序号列、unnamed列、0号序列可以去除 ###

In [ ]:
original_data.info()

### 由此暂时可以得出需要删除的列：中类名称、菜品标签、服务费分摊收入、unnamed、商品销售千次、销售数量占比 ###
### 并且需要用到的数据当中暂无缺失值 ###

In [ ]:
original_data["菜品名称"].value_counts()

### 由此可以评估出存在重复的产品名称，需要进行整合：燕麦奶&燕麦奶【拒绝乳糖！】、鸳鸯拿铁&鸳鸯拿铁【茶与啡的故事】、美式咖啡【醒脑提神】&美式咖啡【仅套餐】&美式【醒脑提神】&美式咖啡（单品规格）【醒脑提神】     、【特惠拿铁+3.99多一杯】双杯拿铁组合！&【特惠3.99多一杯】双杯拿铁组合&【3.99多一杯】双倍暖心拿铁组合！ 、【1.99多一杯】双倍醒神美式组合&【特惠1.99多一杯】双杯美式组合！&美式咖啡（提神醒脑）500ml 、广式点心+咖啡【特色广式搭配！】、广式下午茶套餐 、广式点心套餐【特色广式点心】、收臧福利！珍珠糯米鸡  等 ###

###### 需要就行清理的表格数据：
 - 合并重复项目
 - 删除无关序列
   

###### 删除无关序列



In [ ]:
cleaned_data = original_data.copy()
cleaned_data.head()

In [ ]:
cleaned_data.drop(["中类名称/编码","菜品标签","服务费分摊收入","Unnamed: 14","Unnamed: 15","Unnamed: 16","商品销售千次","销售数量占比"],axis = 1,inplace = True)
cleaned_data.drop([0],inplace = True)
cleaned_data.drop([1],axis=0,inplace = True)
cleaned_data

##### 把“-”替换成NaN，和菜品平均销售价格

In [ ]:
cleaned_data["单个成本/元"] = cleaned_data["单个成本/元"] = cleaned_data["单个成本/元"].replace({"-" : "NaN"})
cleaned_data


##### 逐项更正类似品类名称

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"燕麦奶【拒绝乳糖！】" : "燕麦奶"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "燕麦奶【拒绝乳糖！】"]))

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"鸳鸯拿铁【茶与啡的故事】" : "鸳鸯拿铁"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "鸳鸯拿铁【茶与啡的故事】"]))

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"美式咖啡【仅套餐】" : "美式咖啡【醒脑提神】","美式【醒脑提神】":"美式咖啡【醒脑提神】","美式咖啡（单品规格）【醒脑提神】":"美式咖啡【醒脑提神】"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "美式咖啡【仅套餐】"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "美式【醒脑提神】"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "美式咖啡（单品规格）【醒脑提神】"]))


In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"【特惠3.99多一杯】双杯拿铁组合" : "【特惠拿铁+3.99多一杯】双杯拿铁组合！","【3.99多一杯】双倍暖心拿铁组合！" : "【特惠拿铁+3.99多一杯】双杯拿铁组合！"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "【3.99多一杯】双倍暖心拿铁组合！"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "【特惠3.99多一杯】双杯拿铁组合"]))

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"【特惠1.99多一杯】双杯美式组合！" : "【1.99多一杯】双倍醒神美式组合","美式咖啡（提神醒脑）500ml" : "美式咖啡【醒脑提神】"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "【特惠1.99多一杯】双杯美式组合！"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "美式咖啡（提神醒脑）500ml"]))

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"广式点心+咖啡【特色广式搭配！】" : "广式下午茶套餐","广式点心套餐【特色广式点心】" : "广式下午茶套餐"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "广式点心套餐【特色广式点心】"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "广式点心+咖啡【特色广式搭配！】"]))

##### 下一步是检查类似元素是否替换完成

In [ ]:
cleaned_data["菜品名称"].value_counts()

#### 检查过后还存在类似项目：
- 大红袍奶茶&大红袍奶茶【醇厚甜香】& 大红袍红茶拿铁&大红袍拿铁【醇厚甜香】&大红袍奶茶（茶香四溢）400ml
- 牛奶【健康必备！】&牛奶&全脂牛奶【健康必备！】 
- 热橙美式【满口维C】&热橙美式
- 点新客收藏0.1r送挂耳包&新客收藏0.1r送挂耳包
- 拿铁咖啡（点开有更多选择哦~） & 意式拿铁咖啡【丝滑浓郁】&意式拿铁咖啡【点开有更多选择！】&拿铁&拿铁【丝滑浓郁】

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"大红袍奶茶【醇厚甜香】" : "大红袍奶茶","大红袍红茶拿铁" : "大红袍奶茶","大红袍拿铁【醇厚甜香】" : "大红袍奶茶","大红袍奶茶（茶香四溢）400ml" : "大红袍奶茶"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "大红袍奶茶【醇厚甜香】"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "大红袍红茶拿铁"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "大红袍拿铁【醇厚甜香】"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "大红袍奶茶（茶香四溢）400ml"]))

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"牛奶" : "牛奶【健康必备！】","全脂牛奶【健康必备！】" : "牛奶【健康必备！】"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "牛奶"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "全脂牛奶【健康必备！】"]))


In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"牛奶" : "牛奶【健康必备！】","全脂牛奶【健康必备！】" : "牛奶【健康必备！】"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "牛奶"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "全脂牛奶【健康必备！】"]))

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"热橙美式" : "热橙美式【满口维C】"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "热橙美式"]))

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"点新客收藏0.1r送挂耳包" : "新客收藏0.1r送挂耳包"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "点新客收藏0.1r送挂耳包"]))

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].replace({"拿铁咖啡（点开有更多选择哦~）" : "意式拿铁咖啡【丝滑浓郁】","意式拿铁咖啡【点开有更多选择！】" : "意式拿铁咖啡【丝滑浓郁】","拿铁" : "意式拿铁咖啡【丝滑浓郁】","拿铁【丝滑浓郁】" : "意式拿铁咖啡【丝滑浓郁】"})
print(len(cleaned_data[cleaned_data["菜品名称"] == "拿铁咖啡（点开有更多选择哦~）"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "意式拿铁咖啡【点开有更多选择！】"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "拿铁"]))
print(len(cleaned_data[cleaned_data["菜品名称"] == "拿铁【丝滑浓郁】"]))

##### 再次进行检查

In [ ]:
cleaned_data["菜品名称"].value_counts()

##### 将菜品名称的数据类型转换为字符串

In [ ]:
cleaned_data["菜品名称"] = cleaned_data["菜品名称"].astype(str)
cleaned_data["菜品名称"]

##### 再次检查表格各项数据的数据类型

In [ ]:
cleaned_data.info()

- 发现单个成本/元还需要转换成浮点数
- 收入占比需要转换为浮点数后转化为百分比列
- 销售金额构成需要更换成浮点数
- 退菜率需要更换成浮点数后转化为百分比列

In [ ]:
cleaned_data["销售金额构成"] = cleaned_data["销售金额构成"].astype(float)
cleaned_data["销售金额构成"]

In [ ]:
cleaned_data["单个成本/元"] = cleaned_data["单个成本/元"].astype(float)
cleaned_data["单个成本/元"]

In [ ]:
total = cleaned_data["菜品收入"].sum()
cleaned_data["收入占比"] =  cleaned_data["菜品收入"].apply(lambda x: x/total)

In [ ]:
cleaned_data["退菜率"] =  cleaned_data["退菜数量"] / (cleaned_data["销售数量"] + cleaned_data["退菜数量"])
cleaned_data.head(10)

#### 保存清理后的数据

In [ ]:
cleaned_data.head(20)

In [ ]:
cleaned_data.to_csv("美团_3月.csv",index=False)